**Note about define custom apachebeam template and run that template**
- content: build custom apache beam data pipeline flex template and run it with
- reference link: https://github.com/GoogleCloudPlatform/python-docs-samples/tree/473749daea6b1f1d5a6a8826093a970f03ce0517/dataflow/flex-templates/streaming_beam


In [ ]:
#list google cloud services with --filter flag, filter services with name contain [app_name]
#gcloud services list --available take too long
!gcloud services list --available --filter="name~[app_name]" 

In [ ]:
#enable api service: app engine, cloud scheduler, 
gcloud services enable $service_name

In [ ]:
#create cloud storage bucket with command: gsutil mb
export BUCKET="your-gcs-bucket"
gsutil mb gs://$BUCKET

In [ ]:
# create topic and subscription with command: gcloud pubsub
export TOPIC="messages"
export SUBSCRIPTION="$TOPIC"

gcloud pubsub topics create $TOPIC
gcloud pubsub subscriptions create --topic $TOPIC $SUBSCRIPTION

In [ ]:
#create scheduled job to push data to pubsub with job name is positive-rating-publisher, schedule, topic, message-body
gcloud scheduler jobs create pubsub positive-ratings-publisher \
  --schedule="* * * * *" \
  --topic="$TOPIC" \
  --message-body='{"url": "https://beam.apache.org/", "review": "positive"}'
#trigger job with run command
gcloud scheduler jobs run positive-ratings-publisher

#create scheduled job which push message to
gcloud scheduler jobs create pubsub negative-ratings-publisher \
  --schedule="*/2 * * * *" \
  --topic="$TOPIC" \
  --message-body='{"url": "https://beam.apache.org/", "review": "negative"}'
#trigger the job
gcloud scheduler jobs run negative-ratings-publisher

In [ ]:
#make bigquery dataset with bq mk
#bq cli reference: https://cloud.google.com/bigquery/docs/reference/bq-cli-reference
export PROJECT="$(gcloud config get-value project)"
export DATASET="beam_samples"
export TABLE="streaming_beam"

bq mk --dataset "$PROJECT:$DATASET"

In [ ]:
#build apache beam container image with cloud build command

#set config to use kaniki tools as cache tools
gcloud config set builds/use_kaniko True
#build docker container and submit it to container registry
gcloud builds submit --tag "$dataflow_image" .